---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [70]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [71]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

In [95]:
def answer_one():
    spam=spam_data[spam_data['target']==1]
    spam_num=len(spam)
    not_spam_num=len(spam_data)-spam_num
    perc=spam_num/len(spam_data)
    return perc*100#Your answer here

In [96]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [97]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect=CountVectorizer().fit(X_train)
    tokens=vect.get_feature_names()
    longest=max(tokens,key=len)
    
    return longest#Your answer here

In [98]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect=CountVectorizer().fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    clfrNB=MultinomialNB(alpha=0.1)
    clfrNB.fit(X_train_vectorized,y_train)
    predictions=clfrNB.predict(vect.transform(X_test))
    roc=roc_auc_score(y_test,predictions)
    
    return roc#Your answer here

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect=TfidfVectorizer().fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    feature_names=np.array(vect.get_feature_names())
    sorted_tfidf_index=X_train_vectorized.max(0).toarray()[0].argsort()
    
    smallest_index=sorted_tfidf_index[:20]
    smallest_index_name=feature_names[smallest_index]
    smallest_value=X_train_vectorized.max(0).toarray()[0][smallest_index]
    smallest_final_index = np.concatenate((np.sort(smallest_index[smallest_value==min(smallest_value)]) ,smallest_index[smallest_value!=min(smallest_value)]))

    
    largest_index=sorted_tfidf_index[:-21:-1]
    largest_index_name=feature_names[largest_index]
    largest_value=X_train_vectorized.max(0).toarray()[0][largest_index]
    largest_final_index = np.concatenate((np.sort(largest_index[largest_value==max(largest_value)]) ,largest_index[largest_value!=max(largest_value)]))

    smallest=pd.Series(smallest_value, index=feature_names[smallest_final_index])
    largest=pd.Series(largest_value, index=feature_names[largest_final_index])
    return (smallest, largest)#Your answer here

In [51]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():
    vect=TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    clfrNB=MultinomialNB(alpha=0.1)
    clfrNB.fit(X_train_vectorized,y_train)
    predictions=clfrNB.predict(vect.transform(X_test))
    roc=roc_auc_score(y_test,predictions)
    return roc #Your answer here

In [12]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [25]:
def answer_six():
    spam=spam_data[spam_data['target']==1]
    spam['length']=spam['text'].apply(len)
    not_spam=spam_data[spam_data['target']==0]
    not_spam['length']=not_spam['text'].apply(len)
    spam_mean=np.mean(spam['length'])
    not_spam_mean=np.mean(not_spam['length'])
    return (not_spam_mean, spam_mean)#Your answer here

In [26]:
answer_six()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [52]:
from sklearn.svm import SVC

def answer_seven():
    vect=TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    len_chars_X_train=[]
    len_chars_X_test=[]
    
    for x in X_train:
        len_chars_X_train.append(len(x))
    for x in X_test:
        len_chars_X_test.append(len(x))
    
    new_X_train_vectorized=add_feature(X_train_vectorized, len_chars_X_train)
    new_X_test_vectorized=add_feature(X_test_vectorized,len_chars_X_test)
    
    clfrSVM=SVC(C=10000)
    clfrSVM.fit(new_X_train_vectorized, y_train)
    
    predictions=clfrSVM.predict(new_X_test_vectorized)
    
    auc=roc_auc_score(y_test,predictions)
    return auc#Your answer here

In [53]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [64]:
def answer_eight():
    spam=spam_data[spam_data['target']==1]
    not_spam=spam_data[spam_data['target']==0]
    
    return (not_spam['text'].str.count('\d').mean(), spam['text'].str.count('\d').mean()) #Your answer here

In [65]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [74]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vect=TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    len_chars_X_train=[]
    len_chars_X_test=[]
    
    for x in X_train:
        len_chars_X_train.append(len(x))
    for x in X_test:
        len_chars_X_test.append(len(x))
    
    dig_count_X_train=X_train.str.count('\d')
    dig_count_X_test=X_test.str.count('\d')
    
    new_X_train_vectorized=add_feature(X_train_vectorized, len_chars_X_train)
    new_X_test_vectorized=add_feature(X_test_vectorized,len_chars_X_test)
    
    newer_X_train_vectorized=add_feature(new_X_train_vectorized, dig_count_X_train)
    newer_X_test_vectorized=add_feature(new_X_test_vectorized, dig_count_X_test)
    
    clfrLR=LogisticRegression(C=100)
    clfrLR.fit(newer_X_train_vectorized, y_train)
    predictions=clfrLR.predict(newer_X_test_vectorized)
    
    auc=roc_auc_score(y_test, predictions)
    return auc#Your answer here

In [75]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [78]:
def answer_ten():
    spam=spam_data[spam_data['target']==1]
    not_spam=spam_data[spam_data['target']==0]
    
    
    return (not_spam['text'].str.count('\W').mean(), spam['text'].str.count('\W').mean())  #Your answer here

In [79]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [115]:
def answer_eleven():
    vect=CountVectorizer(analyzer='char_wb',min_df=5, ngram_range=(2,5)).fit(X_train)
    X_train_vectorized=vect.transform(X_train)
    X_test_vectorized=vect.transform(X_test)
    len_chars_X_train=[]
    len_chars_X_test=[]
    
    for x in X_train:
        len_chars_X_train.append(len(x))
    for x in X_test:
        len_chars_X_test.append(len(x))
    
    dig_count_X_train=X_train.str.count('\d')
    dig_count_X_test=X_test.str.count('\d')
    
    oth_count_X_train=X_train.str.count('\W')
    oth_count_X_test=X_test.str.count('\W')
    
    new_X_train_vectorized=add_feature(X_train_vectorized, len_chars_X_train)
    new_X_test_vectorized=add_feature(X_test_vectorized,len_chars_X_test)
    
    newer_X_train_vectorized=add_feature(new_X_train_vectorized, dig_count_X_train)
    newer_X_test_vectorized=add_feature(new_X_test_vectorized, dig_count_X_test)
    
    newest_X_train_vectorized=add_feature(newer_X_train_vectorized, oth_count_X_train)
    newest_X_test_vectorized=add_feature(newer_X_test_vectorized, oth_count_X_test)
    
    columns=np.array(vect.get_feature_names())
    new_columns=np.append(columns,['length_of_doc', 'digit_count','non_word_char_count'])
    
    newest_X_train_vectorized.columns=new_columns.tolist()
    newest_X_test_vectorized.columns=new_columns.tolist()
    
    clfrLR=LogisticRegression(C=100)
    clfrLR.fit(newest_X_train_vectorized, y_train)
    predictions=clfrLR.predict(newest_X_test_vectorized)
    
    auc=roc_auc_score(y_test, predictions)
    
    sorted_coef_index=clfrLR.coef_[0].argsort()
    
    smallest=new_columns[sorted_coef_index[:10]].tolist()
    largest=new_columns[sorted_coef_index[:-11:-1]].tolist()
    return (auc, smallest, largest) #Your answer here

In [116]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])